In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tmdb-box-office-prediction/train.csv')
sub = pd.read_csv('/kaggle/input/tmdb-box-office-prediction/sample_submission.csv')
test = pd.read_csv('/kaggle/input/tmdb-box-office-prediction/test.csv')

In [ ]:
pd.set_option('display.max_columns', 500)
train.head(2)

In [ ]:
test.head(2)

In [ ]:
sub

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train['release_date'] = pd.to_datetime(train['release_date'])

In [ ]:
train['release_year'] = train['release_date'].dt.year

In [ ]:
train.isnull().sum()

In [ ]:
train.dropna(inplace=True)

In [ ]:
train.duplicated().sum()

In [ ]:
train = train.drop(['belongs_to_collection','homepage','imdb_id','release_date','original_title',
                    'poster_path','spoken_languages','status','tagline'],axis=1)

In [ ]:
train.head(2)

In [ ]:
import ast

In [ ]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [ ]:
train['genres'] = train['genres'].apply(convert)
train['production_companies'] = train['production_companies'].apply(convert)
train['production_countries'] = train['production_countries'].apply(convert)
train['Keywords'] = train['Keywords'].apply(convert)

In [ ]:
def convert_cast(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L
        

In [ ]:
train['cast'] = train['cast'].apply(convert_cast)

In [ ]:
train['actor_1'] = train['cast'].apply(lambda x:x[0:1])
train['actor_2'] = train['cast'].apply(lambda x:x[1:2])
train['actor_3'] = train['cast'].apply(lambda x:x[2:3])

In [ ]:
def convert_crew(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [ ]:
train['director'] = train['crew'].apply(convert_crew)

In [ ]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(' ', ''))
    return L1

In [ ]:
train['genres'] = train['genres'].apply(collapse)
train['Keywords'] = train['Keywords'].apply(collapse)
train['cast'] = train['cast'].apply(collapse)
train['crew'] = train['crew'].apply(collapse)
train['actor_1'] = train['actor_1'].apply(collapse)
train['actor_2'] = train['actor_2'].apply(collapse)
train['actor_3'] = train['actor_3'].apply(collapse)

In [ ]:
train.head(2)

In [ ]:
train.drop(['crew','overview','cast'],axis=1,inplace=True)

In [ ]:
train['genres'] = train['genres'].apply(lambda x: " ".join(x))
train['production_companies'] = train['production_companies'].apply(lambda x: " ".join(x))
train['production_countries'] = train['production_countries'].apply(lambda x: " ".join(x))
train['Keywords'] = train['Keywords'].apply(lambda x: " ".join(x))
train['actor_1'] = train['actor_1'].apply(lambda x: " ".join(x))
train['actor_2'] = train['actor_2'].apply(lambda x: " ".join(x))
train['actor_3'] = train['actor_3'].apply(lambda x: " ".join(x))
train['director'] = train['director'].apply(lambda x: " ".join(x))

In [ ]:
train.head(2)

In [ ]:
df_num = train.select_dtypes([np.number])
df_cat = train.select_dtypes(['object'])

In [ ]:
df_num.head(2)

In [ ]:
df_cat.head(2)

In [ ]:
df_cat1 = pd.get_dummies(df_cat,drop_first=True)

In [ ]:
df_cat1.head(2)

In [ ]:
df = pd.concat([df_num,df_cat1],axis=1)

In [ ]:
df.head(2)

In [ ]:
y = df['revenue']

In [ ]:
x = df.drop(['revenue'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=8)

In [ ]:
lm=LinearRegression()
lm.fit(x_train,y_train)
y_pred = lm.predict(x_test)
r_squared=r2_score(y_test,y_pred)
r_squared